# PHASE 1


### Phase 1 Plan
Imports & Environment Setup

Train SAC Agent

Save the Model

Load the Model

Evaluate the Trained Agent

Plot Training Results

Imports & Environment Setup

In [43]:
import gymnasium as gym
import panda_gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

from stable_baselines3.common.env_util import make_vec_env
from gymnasium.wrappers import FlattenObservation

### Train SAC Agent

In [41]:
from stable_baselines3.common.vec_env import VecEnvWrapper

class SafeVecEnv(VecEnvWrapper):
    def __init__(self, venv):
        super().__init__(venv)

    def reset(self):
        obs = self.venv.reset()
        return self._sanitize_obs(obs)

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step_wait(self):
        obs, rewards, dones, infos = self.venv.step_wait()
        return self._sanitize_obs(obs), rewards, dones, infos

    def _sanitize_obs(self, obs):
        if isinstance(obs, dict):
            obs = np.concatenate([v.ravel() for v in obs.values()], axis=0)
            obs = obs.reshape((1, -1)).astype(np.float32)
        elif isinstance(obs, list):
            obs = np.array(obs, dtype=np.float32)
        return obs


In [ ]:
# Create and wrap the training environment
# Training environment
env = gym.make("PandaPickAndPlace-v3")
env = Monitor(env)
env = FlattenObservation(env)
env = DummyVecEnv([lambda: env])
env = VecMonitor(env)
env = SafeVecEnv(env) 


# Evaluation environment
eval_env = gym.make("PandaPickAndPlace-v3")
eval_env = Monitor(eval_env)
eval_env = FlattenObservation(eval_env)
eval_env = DummyVecEnv([lambda: eval_env])
eval_env = VecMonitor(eval_env)
eval_env = SafeVecEnv(eval_env)  


# # Define evaluation callback
# eval_callback = EvalCallback(
#     eval_env,
#     best_model_save_path='./best_model/',
#     log_path='./logs/',
#     eval_freq=5000,
#     deterministic=True,
#     render=False
# )

# # Create and train the model
# model = model = SAC("MlpPolicy", env, verbose=1, tensorboard_log="./sac_panda_tensorboard/")
# model.learn(total_timesteps=100_000, callback=eval_callback)


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


: 

Goal-conditioned environments (like PandaPickAndPlace) often require special wrappers like GoalEnvWrapper depending on the training method. If you're planning to use HER (Hindsight Experience Replay), you might want to use:

from stable_baselines3.her import HER
from stable_baselines3 import SAC

model = HER(
    "MultiInputPolicy",
    env,
    SAC,
    n_sampled_goal=4,
    goal_selection_strategy="future",
    verbose=1,
    tensorboard_log="./sac_panda_tensorboard/"
)

In [37]:
print("Sample env.obs_space:", env.observation_space)
sample_obs = env.reset()
print("Sample obs (training):", sample_obs)
print("Shape:", sample_obs.shape, "Type:", type(sample_obs))

Sample env.obs_space: Box(-10.0, 10.0, (25,), float32)
Sample obs (training): [[-7.5920761e-02  1.3379993e-01  2.0000000e-02 -1.7616393e-02
  -3.1456467e-02  1.8847208e-01  3.8439669e-02 -2.1944723e-12
   1.9740014e-01  0.0000000e+00 -0.0000000e+00  0.0000000e+00
   0.0000000e+00 -7.5920761e-02  1.3379993e-01  2.0000000e-02
   0.0000000e+00 -0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00]]
Shape: (1, 25) Type: <class 'numpy.ndarray'>


In [38]:
from stable_baselines3.common.callbacks import BaseCallback

class ObservationSanityCallback(BaseCallback):
    def __init__(self):
        super().__init__()

    def _on_step(self):
        obs = self.model._last_obs
        if isinstance(obs, dict):
            print("❌ ERROR: Observation is a dict, not np.array!")
        elif not isinstance(obs, np.ndarray):
            print("❌ ERROR: Observation is not np.array. It's", type(obs))
        elif obs.dtype != np.float32:
            print("❌ ERROR: Observation dtype is not float32. It's", obs.dtype)
        return True


In [39]:
# Create and train the model
model = model = SAC("MlpPolicy", env, verbose=1, tensorboard_log="./sac_panda_tensorboard/")


Using cuda device


In [40]:
model.learn(total_timesteps=100_000, callback=[eval_callback, ObservationSanityCallback()])


Logging to ./sac_panda_tensorboard/SAC_5


TypeError: 

In [ ]:
model.learn(total_timesteps=100_000, callback=eval_callback)